In [ ]:
#https://www.sparkfun.com/products/13322

In [19]:
GPIO.cleanup() #for debugging

In [ ]:
import RPi.GPIO as GPIO
import time
import spiUtils as su
import tweepy
import os
import datetime as dt

pump_pin=18
light_pin=19
power_moist=20
pump_count=0

DARK=500
DRY=800

# twitter personal details - hidden for interview
consumer_key = "-----------------"
consumer_secret ="-----------------"
access_token ="-----------------"
access_token_secret ="-----------------"
auth = tweepy.OAuthHandler(consumer_key, consumer_secret) 
auth.set_access_token(access_token, access_token_secret) 
api = tweepy.API(auth) 


GPIO.setmode(GPIO.BCM) #GPIO Setup

#Water Pump Setup
GPIO.setup(pump_pin, GPIO.OUT)
GPIO.output(pump_pin,1)

#Soil Moisture Sensor Setup
GPIO.setup(power_moist, GPIO.OUT)
GPIO.output(power_moist,1)

#Plant Growth Light Setup
GPIO.setup(light_pin, GPIO.OUT)

def p(length): #Pump Spray burst
    GPIO.output(pump_pin,0)
    time.sleep(length)
    GPIO.output(pump_pin,1)

def p2(length): #Pump Spray Burst
    start = time.time()
    GPIO.output(pump_pin,0)
    while(time.time()<start+length):
            a=0
    GPIO.output(pump_pin,1)
    
def moistRead():
    GPIO.output(power_moist,0) #Power on sensor
    time.sleep(3) #Wait to reduce error
    
    moisture = su.readADC(channel=0)# Collect Moisture Data
    print(moisture)
    
    GPIO.output(power_moist,1) #Power off sensor
    return moisture

def lightRead():
    return su.readADC(channel=1) #Read light sensor
    
while(True):
    #Timestamp Info
    now = time.time()
    hour = (dt.datetime.now().hour +24 -7)%24
    minute = dt.datetime.now().minute
    print(hour,':',minute,".",pump_count)
    
    if(lightRead()<DARK): #if room is dark
        print("dark")
        if(7<=hour and hour<=19): #and it's between 7am and 8pm
            GPIO.output(light_pin,1) # turn on the grow light
            print("light on")
    if(moistRead()<DRY): # if soil is dry
        print("pumping")
        p(3) # pumps for 3 seconds
        pump_count = pump_count + 1
        
    if( hour==12 and minute==0):# every day at noon
        #Prepare Twitter Message
        status = "Moisture: "+str(moistRead())+", Light: "+str(lightRead())
        print("Picture time")
        timestamp = time.time()
        #Take Picture
        picname = "ppic"+str(timestamp)+".jpg"
        os.system("raspistill -n -q 10 -o "+picname)
        #Post to Twitter
        api.update_with_media(picname,status)
        print(status)
    while(time.time()<now+60): #wait a minute (clock cycle)
        time.sleep(0.1)
    GPIO.output(light_pin,0) #LED OFF while state is re-evaluated (imperceptable to people)


    

In [22]:
#Test Pumps
p(5)

In [21]:
#Test Sensors
print("light:",lightRead())
print("moist:",moistRead())

light: 147
792
moist: 792


In [ ]:
#Test Camera and Twitter
# importing the module 
import tweepy 
import time
import os
  
# personal details 
consumer_key ="--------------------------"
consumer_secret ="--------------------------"
access_token ="--------------------------"
access_token_secret ="--------------------------"
  
# authentication of consumer key and secret 
auth = tweepy.OAuthHandler(consumer_key, consumer_secret) 
  
# authentication of access token and secret 
auth.set_access_token(access_token, access_token_secret) 
api = tweepy.API(auth) 

#camera setup

# update the status with image
timestamp = time.time()
picname = "ppic"+str(timestamp)+".jpg"
os.system("raspistill -n -q 10 -o "+picname)
api.update_with_media(picname)



In [25]:
#Test All Components for Posting
status = "Moisture: "+str(moistRead())+", Light: "+str(lightRead())
print("Picture time")
timestamp = time.time()
picname = "ppic"+str(timestamp)+".jpg"
os.system("raspistill -n -q 10 -o "+picname)
api.update_with_media(picname,status)
print(status)

Picture time
Moisture: 837, Light: 881
